In [1]:
import tensorrt as trt
import onnx

In [2]:
trt_logger = trt.Logger(trt.Logger.WARNING)
trt.init_libnvinfer_plugins(trt_logger, namespace="")
builder = trt.Builder(trt_logger)

In [3]:
config = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 16 * (2**30))
config.flags |= 1 << int(trt.BuilderFlag.FP16)
config.default_device_type = trt.DeviceType.GPU

In [4]:
network = builder.create_network(1)
model_onnx = onnx.load("rt-detr_model.onnx")

In [5]:
parser = trt.OnnxParser(network, trt_logger)
parser.parse(model_onnx.SerializeToString())

True

In [6]:
inputs = [network.get_input(i) for i in range(network.num_inputs)]
outputs = [network.get_output(i) for i in range(network.num_outputs)]

for input in inputs:
    batch_size = input.shape[0]

In [7]:
config.set_flag(trt.BuilderFlag.FP16)
# config.set_flag(trt.BuilderFlag.INT8)
engine_bytes = builder.build_serialized_network(network, config)
with open("model.trt", 'wb') as f:
    f.write(bytearray(engine_bytes))